<a href="https://colab.research.google.com/github/raut-sam/starting-with-colab/blob/master/Vine-quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experience with iris dataset using tf.keras & tensorflow

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [0]:
#pip install tensorflow==1.14.0

In [0]:
print(tf.__version__)

## Data download and dataset creation witout tf.data

In [0]:
ds_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
column_names = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
                    'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']

#species = np.array(['Setosa', 'Versicolor', 'Virginica'], dtype=np.object)

## Load data

In [0]:
dataset_path = tf.keras.utils.get_file(ds_url.split('/')[-1], ds_url)
dataset_csv = pd.read_csv(dataset_path, names=column_names,
                              na_values="?", comment='\t',
                              sep=";", skipinitialspace=True, header=0)

dataset_final = dataset_csv[['alcohol','chlorides','quality']]



In [0]:
bins = (2,6.5,8)
group_names = ['bad','good']
categories = pd.cut(dataset_final['quality'], bins, labels = group_names)
#dataset_final['quality'] = categories
dataset_final['quality'] = categories.replace({ "bad":0, "good" : 1})

In [0]:
train_dataset = dataset_final.sample(frac=0.8, random_state=0)
test_dataset = dataset_final.drop(train_dataset.index)

In [0]:
train_dataset

In [0]:
dataset_final['quality'] 

In [0]:
#train_dataset['quality'].value_counts()
#c_train_dataset = train_dataset['quality'].replace({ 3:0, 4 : 1, 5 : 2, 6 : 3,7:4, 8:5})

In [0]:
#c_train_dataset.value_counts()

In [0]:
train_dataset = train_dataset.apply(pd.to_numeric)
test_dataset = test_dataset.apply(pd.to_numeric)

In [0]:
#test_dataset["quality"].size()
#train_dataset['quality'] = train_dataset['quality'].apply(str)
###train_dataset['quality'] = train_dataset['quality'].astype(str)

In [0]:
#test_dataset
train_features, train_categories = train_dataset, train_dataset.pop("quality")
test_features, test_categories = test_dataset, test_dataset.pop("quality")

In [0]:
#final_categories = categories.replace({ 3:0, 4 : 1, 5 : 2, 6 : 3,7:4, 8:5})
#final_test_categories = test_categories.replace({ 3:0, 4 : 1, 5 : 2, 6 : 3,7:4, 8:5})

In [0]:
#train_dataset['quality'].value_counts()
#categories.value_counts()
#categories = categories.astype(str)

In [0]:
#categories.value_counts()
#len(final_categories)
train_categories.value_counts()

In [0]:
y_categorical = tf.contrib.keras.utils.to_categorical(train_categories,num_classes=2)
test_y_categorical = tf.contrib.keras.utils.to_categorical(test_categories, num_classes=2)
#y_categorical_test = tf.contrib.keras.utils.to_categorical(test_categories, num_classes=len(categories))

In [0]:
#train_dataset["quality"].unique
y_categorical

## Load data from BigQuery [optional]

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
project_id = '' #is needed use a cloud project id
df_train = pd.io.gbq.read_gbq('''SELECT * FROM [socialagilelearning:iris.training]''', project_id=project_id, verbose=False)
df_test = pd.io.gbq.read_gbq('''SELECT * FROM [socialagilelearning:iris.test]''', project_id=project_id, verbose=False)

In [0]:
categories='Plants'
train_plantfeatures, train_categories = df_train, df_train.pop(categories)
test_plantfeatures, test_categories = df_test, df_test.pop(categories)

In [0]:
y_categorical = tf.contrib.keras.utils.to_categorical(train_categories, num_classes=3)
y_categorical_test = tf.contrib.keras.utils.to_categorical(test_categories, num_classes=3)

##  Build  the Dataset
from_tensor_slices

To build the dataset we will use tf.data.Dataset set of elements. 

In [0]:
#dataset = tf.data.Dataset.from_tensor_slices((train_features.values, y_categorical))
dataset = tf.data.Dataset.from_tensor_slices((train_features.values, train_categories.values))
dataset = dataset.batch(32)
dataset = dataset.shuffle(1000)
dataset = dataset.repeat()

In [0]:
#dataset_test = tf.data.Dataset.from_tensor_slices((test_features.values, test_y_categorical))
dataset_test = tf.data.Dataset.from_tensor_slices((test_features.values, test_categories.values))

dataset_test = dataset_test.batch(32)
dataset_test = dataset_test.shuffle(1000)
dataset_test = dataset_test.repeat()

## Build  the Model

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(32, activation='relu', input_dim=2),
  tf.keras.layers.Dense(64,activation='relu'),
  tf.keras.layers.Dense(64,activation='relu'),
  #tf.keras.layers.Dense(2, activation=tf.nn.softmax),
    tf.keras.layers.Dense(2, activation=tf.nn.sigmoid),
])

model.summary()

In [0]:
#op = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#optimizer='adam'
##model.compile(loss='categorical_crossentropy',
##              optimizer='adam',
##              metrics=['accuracy'])



In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

## Train the Model

In [0]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)

model.fit(dataset, steps_per_epoch=32, epochs=300, callbacks=[early_stop], verbose=1)

## Eval the model

In [0]:
loss, accuracy = model.evaluate(dataset_test, steps=32)

print("loss:%f"% (loss))
print("accuracy: %f"%   (accuracy))

## Use the model

If you need to test another specie, you can modify the **new_specie** array.

In [0]:
new_vine = np.array([[10.0, 0.065]])
prediction = model.predict_classes(new_vine)
prediction
if prediction[0] == 1:
  print("Good")
else:
  print("Bad")

## Save the model

In [0]:
!mkdir model

In [0]:
tf.keras.models.save_model(
    model,
    "./model/iris_model.h5",
    overwrite=True,
    include_optimizer=True
)

In [0]:
new_model = tf.keras.models.load_model("./model/iris_model.h5")

xarray2 = np.array([7.9,3.8,6.4,2.0])

pred = np.around(new_model.predict(np.expand_dims(xarray2, axis=0))).astype(np.int)[0]

print(pred)

print("That means it's a %s" % species[pred.astype(np.bool)][0])

## Visualize the Graph

In [0]:
graph = tf.get_default_graph()

In [0]:
# Let's visualize our graph!
# Tip: to make your graph more readable you can add a
# name="..." parameter to the individual Ops.

# src: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb
# requeried if is not importated before
# import tensorflow as tf
# import numpy as np

from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [0]:
show_graph(graph)